# 束搜索

束搜索是介于暴力枚举和贪心算法之间的，求解最终每一个时间步输出的方法

超参数
- 束宽 num_beam
- 惩罚参数 $\alpha$

算法流程：
1. 对于第一个时间步，获取概率最大的 num_beam 个词元，作为接下来的开始词元
2. 接下来，以第一个时间步生成的k个词元，向后进行 贪心搜索，对于每一个序列，直到模型输出\<eos\>
3. 最终输出 (num_beam x 序列长度) 个序列
4. 最后根据评分函数选取评分最高的序列
当k=1时，束搜索退化为贪心搜索
$$
L = \frac{\sum log(P_i)}{长度^{\alpha}}
$$

一些其他参数及其意义：

**top_k:**

限制选词集合，由于模型会对每一个词元输出概率，直接从整个vocab里采样可能不好，我们可以仅保留k个概率最高的词元，**然后重新对输出词元的概率分布归一化**

**top_p:**

类似与top_k:，也是把一些低概率的词元的概率设置为0，而计算方法是，按概率降序排序之后，开始逐个累加，直到找到Sum >= top_q 时对应的下标i，把i后的所有词元概率设置为0

**temperature**

不同与上述两个参数的作用，temperature用于放大或缩小原有概率，具体来说 对于每一个词元的概率P，有 $ P^{'} = \frac{P}{T}$

当T大于1时，概率变得比较平滑，增加了随机性

当小于1时，概率变的集中，生成更确定的结果

最后的概率 $P = SoftMax(P^{'})$





In [30]:
import torch.nn as nn
import torch

def sample_top_p(probs, p):
    for batch in probs:
        cumulative_sum  = 0.0
        index = 0
        batch.sort(descending=True)
        for i, prob in enumerate(batch):
            cumulative_sum  += prob
            if cumulative_sum  >= p:
                index = i
                break
        batch[index+1:] = torch.full_like(batch[index+1:], 1e-8)
        batch = nn.functional.softmax(batch, dim=0)
    return probs


In [31]:
nn.functional.softmax(torch.randn(size=(10,10)), dim=1)

tensor([[0.0244, 0.1757, 0.0432, 0.0566, 0.0613, 0.0909, 0.1736, 0.2569, 0.0984,
         0.0190],
        [0.0483, 0.0402, 0.1022, 0.0881, 0.0553, 0.0815, 0.1876, 0.0519, 0.0130,
         0.3319],
        [0.1384, 0.0804, 0.0637, 0.3584, 0.0212, 0.0755, 0.0244, 0.0975, 0.0650,
         0.0754],
        [0.0813, 0.0542, 0.0231, 0.0356, 0.1064, 0.2528, 0.1310, 0.0485, 0.0268,
         0.2402],
        [0.0826, 0.0136, 0.0735, 0.0404, 0.2627, 0.0279, 0.0539, 0.0314, 0.0294,
         0.3846],
        [0.0151, 0.0609, 0.0373, 0.0585, 0.2911, 0.0073, 0.1144, 0.0163, 0.0866,
         0.3124],
        [0.1330, 0.1218, 0.1134, 0.0867, 0.0250, 0.2074, 0.1353, 0.1072, 0.0082,
         0.0621],
        [0.2312, 0.0218, 0.0236, 0.0562, 0.0800, 0.0977, 0.0299, 0.1982, 0.2322,
         0.0291],
        [0.0721, 0.0914, 0.0337, 0.0506, 0.1588, 0.0903, 0.0460, 0.0222, 0.3205,
         0.1145],
        [0.0713, 0.0300, 0.1515, 0.0325, 0.2994, 0.0375, 0.0136, 0.1570, 0.0467,
         0.1605]])

In [32]:
probs = torch.tensor([[0.1, 0.3, 0.4, 0.2],
                      [0.5, 0.2, 0.2, 0.1]], dtype=torch.float32)
p = 0.7

result = sample_top_p(probs, p)
print(result)

tensor([[1.0000e-01, 3.0000e-01, 4.0000e-01, 1.0000e-08],
        [5.0000e-01, 2.0000e-01, 1.0000e-08, 1.0000e-08]])
